In [ ]:
import pandas as pd
from tqdm import tqdm
from pathlib import Path
import re
import plotly.express as px
import glob
from collections import Counter
from collections import defaultdict

# 1) Häufigkeiten der Wortfelder für Romankorpus berechnen

## Daten einlesen

In [ ]:
#lemmatisiertes Korpus einlesen
df = pd.read_csv("data/roman_corpus_lemmatisiert.csv", sep=";")
len(df)

In [ ]:
# Listen wiederherstellen (weil als String eingelesen)
for i in tqdm(range(len(df))):
    lemmas = str(df["all_lemmas"].loc[i])
    lemmas = re.sub(r"'", "", lemmas)
    lemmas = re.sub(r"\[", "", lemmas)
    lemmas = re.sub(r"\]", "", lemmas)
    lemmas = lemmas.split(", ")
    df["all_lemmas"].loc[i] = lemmas
    #print(type(lemmas))

## Berechnung der Anzahl jeweiliger Wortfelder

In [ ]:
# Function for getting wordlist
def process_wordlist(Pfad):
    wordlist_df = pd.read_csv(Pfad, sep=";", encoding="utf-8")
    words = list(wordlist_df["Word"])
    
    return words



In [ ]:
def count_words_in_lemmalist(wordlist, counter):
    words_counter = 0
    for word in wordlist:
        words_counter += counter[word]
    return words_counter

In [ ]:
files = glob.glob("data/Wordlists_from_fasttext/*cleaned.csv")

In [ ]:
for file in files:
    
    wortfeld = file.split("/")[1].split("_")[0]
    #print(wortfeld)
    print("Prozessiere: " + wortfeld + " " + str(files.index(file)))
    
    
    string = "anteil_" + wortfeld
    df[string] = 0
    wortfeld_woerter = process_wordlist(file)
    for i in tqdm(range(len(df))):
        lemmas = df["all_lemmas"].loc[i]
        all_words_count = len(lemmas)
        counter = Counter(lemmas)
    
        wortfeld_counter = count_words_in_lemmalist(wortfeld_woerter, counter)
        df[string].loc[i] = wortfeld_counter/all_words_count
    

## Häufigkeiten für konkret/abstrakt summieren

In [ ]:
df["anteil_konkret"] = df["anteil_Ornamente"] + df["anteil_Dekorationsgegenstaende"] \
                    + df["anteil_Bauwerke"] + df["anteil_Gebaeudeteile"] + \
                    df["anteil_Zimmer"] + df["anteil_Moebel"] + \
                    df["anteil_Heimtextilien"] + df["anteil_Haushaltsgegenstaende"] + \
                    df["anteil_Kleidung"] + df["anteil_Kleidungsteile"] + \
                    df["anteil_Aufmachung"] + df["anteil_aussehensspezifisch"] \
                    + df["anteil_Taschen"] + df["anteil_Stoffe"] +   \
                    df["anteil_formspezifisch"] + \
                    df["anteil_helligkeitsspezifisch"] + df["anteil_oberflaechenspezifisch"] + \
                    df["anteil_Holz"] +df["anteil_Koerperteile"] + df["anteil_Farben"] + \
                    df["anteil_Gruenflaechen"] + df["anteil_Gerueche"] + df["anteil_Pflanzen"] + \
                    df["anteil_Pflanzenteile"] + df["anteil_Gartenanlagen"] + \
                    df["anteil_Wege"] + df["anteil_Waelder"] + \
                    df["anteil_Bilder"] + df["anteil_Schmuck"]+ df["anteil_Kunstobjekte"] + \
                    df["anteil_Muster"]

In [ ]:
df["anteil_abstrakt"] = df["anteil_Emotionen"] + df["anteil_Charaktereigenschaften"]

## Berechnung Anzahl einiger Grundwörter

In [ ]:
Grundfarben = ["rot", "blau", "grün"]
Grundkleidung = ["Kleid", "Mantel", "Hut"]
Grundkoerperteile = ["Finger", "Ohr", "Nase"]
wortfelder = [Grundfarben, Grundkleidung, Grundkoerperteile]

In [ ]:
for wortfeld in wortfelder:
   
    string = "anteil_" + str(wortfeld)
    
    df[string] = 0
    
    
    for i in tqdm(range(len(df))):
        lemmas = df["all_lemmas"].loc[i]
        
        all_words_count = len(lemmas)
        counter = Counter(lemmas)
    
        wortfeld_counter = count_words_in_lemmalist(wortfeld, counter)
        df[string].loc[i] = wortfeld_counter/all_words_count

## Speichern als csv-Datei

In [ ]:
df_without_lemmas = df.copy()
df_without_lemmas = df_without_lemmas.drop(columns=["all_lemmas"])
df_without_lemmas.to_csv("data/roman_corpus_word_frequencies.csv", sep=";")

# 2) Häufigkeiten der Wortgruppen "abstrakt" und "konkret" für Romankorpus berechnen unter Berücksichtigung der Redeform

In [ ]:
konkrete_wortfelder = ["Ornamente", "Dekorationsgegenstaende", "Bauwerke", "Gebaeudeteile",  "Zimmer",\
                       "Moebel", "Heimtextilien", "Haushaltsgegenstaende", "Kleidung", "Kleidungsteile" ,\
                       "Aufmachung", "aussehensspezifisch", "Taschen", "Stoffe", "formspezifisch", \
                       "helligkeitsspezifisch", "oberflaechenspezifisch", "Koerperteile", "Farben",\
                      "Pflanzen", "Pflanzenteile", "Schmuck", "Gruenflaechen", "Bilder", "Gartenanlagen",\
                      "Waelder", "Kunstobjekte", "Wege", "Gerueche", "Muster", "Holz"] 
konkrete_woerter = []
for wortfeld in konkrete_wortfelder:
    file = "data/Wordlists_from_fasttext/" + wortfeld + "_from_fasttext_cleaned.csv"
    konkrete_woerter += process_wordlist(file)

In [ ]:
charaktereigenschaften = process_wordlist("data/Wordlists_from_fasttext/Charaktereigenschaften_from_fasttext_cleaned.csv")
emotionen = process_wordlist("data/Wordlists_from_fasttext/Emotionen_from_fasttext_cleaned.csv")
abstrakte_woerter = charaktereigenschaften + emotionen

In [ ]:
# Dictionary erstellen, in dem Wörter aus Wortlisten nach abstrakt/konkret

def def_value():
    return "Not Present"
words_dict = defaultdict(def_value)
for word in konkrete_woerter:
    words_dict[word] = "konkr"

for word in abstrakte_woerter:
    words_dict[word] = "abst"
#words_dict

In [ ]:
wortfelder = ["Ornamente", "Dekorationsgegenstaende", "Bauwerke", "Gebaeudeteile",  "Zimmer",\
                       "Moebel", "Heimtextilien", "Haushaltsgegenstaende", "Kleidung", "Kleidungsteile" ,\
                       "Aufmachung", "aussehensspezifisch", "Taschen", "Stoffe", "formspezifisch", \
                       "helligkeitsspezifisch", "oberflaechenspezifisch", "Koerperteile", "Farben",\
                      "Pflanzen", "Pflanzenteile", "Schmuck", "Gruenflaechen", "Bilder", "Gartenanlagen",\
                      "Waelder", "Kunstobjekte", "Wege", "Gerueche", "Muster", "Holz", "Charaktereigenschaften", \
             "Emotionen"] 


In [ ]:
wortfelder_dict = defaultdict(def_value)
for wortfeld in wortfelder:
    file = "data/Wordlists_from_fasttext/" + wortfeld + "_from_fasttext_cleaned.csv"
    wortlist = process_wordlist(file)
    for wort in wortlist:
        wortfelder_dict[wort] = wortfeld
#wortfelder_dict

In [ ]:
wortfelder_dict["Glück"]

In [ ]:
df = pd.read_csv("data/roman_corpus_lemmatisiert.csv", sep=";")

In [ ]:
df["ant_narr_abstrakt"] = 0
df["ant_ind_abstrakt"] = 0
df["ant_dir_abstrakt"] = 0
df["ant_rep_abstrakt"] = 0
df["ant_free_abstrakt"] = 0

df["ant_narr_konkret"] = 0
df["ant_ind_konkret"] = 0
df["ant_dir_konkret"] = 0
df["ant_rep_konkret"] = 0
df["ant_free_konkret"] = 0

df["ant_dir_rede"] = 0
df["ant_ind_rede"] = 0
df["ant_rep"]= 0
df["ant_free"] = 0
df["ant_narr"] = 0

In [ ]:
for wortfeld in wortfelder:
    df["anteil_" + wortfeld] = 0
#df

In [ ]:
# Abstconc-Werte als Dictionary
# the dataset used to compute the scores can be downloaded from this website: http://www.schulteimwalde.de/resources/affective-norms.html

dfca = pd.read_csv("data/ratings_lrec16_koeper_ssiw.txt", sep="\t", encoding="utf-8")
dfca_dict = defaultdict(def_value)
for i in range(len(dfca)):
    key = dfca["Word"].loc[i]
    value = dfca["AbstConc"].loc[i]
    dfca_dict[key] = value
#dfca_dict

In [ ]:
df["abstconc"] = 0
df["abstconc_dir"] = 0
df["abstconc_ind"] = 0
df["abstconc_rep"] = 0
df["abstconc_free"] = 0
df["abstconc_narr"] = 0


In [ ]:
# Diese Schleife iteriert über alle Romane des Korpus und liest jeweils den Output aus dem Redewiedergabe-Tagger ein.
# Anschließend wird für jede Redewiedergabe-Art der Anteil der konkreten bzw. abstrakten Wörter ermittelt.
# Auf Basis des Datensatzes von Schulte im Walde und Köper wird zudem ein Konkretheitswert für den jeweiligen Rede-
# wiedergabetyp ermittelt.
# Textpassagen, die mit keiner der Redewiedergabe-Typen getaggt wurden werden als Erzählertext ("narr") aufgefasst.

import os
# Für jeden Roman im Korpus
for i in tqdm(range(429, len(df))):
    filename_dir = "data/output_tagger/outputdir_direct/" + df["filename"].loc[i] + ".tsv"
    filename_ind = "data/output_tagger/outputdir_indirect/" + df["filename"].loc[i] + ".tsv"
    filename_rep = "data/output_tagger/outputdir_reported/" + df["filename"].loc[i] + ".tsv"
    filename_free = "data/output_tagger/outputdir_free/" + df["filename"].loc[i] + ".tsv"
    
    # einzelne Redewiedergabe dfs einlesen und mergen
    if os.path.exists(filename_dir) and os.path.exists(filename_ind) and os.path.exists(filename_rep) and \
    os.path.exists(filename_free):
        
        df_dir = pd.read_csv(filename_dir, sep="\t").reset_index()
        df_ind = pd.read_csv(filename_ind, sep="\t").reset_index()
        df_merg = df_dir.merge(right=df_ind, left_index=True, right_on='index')
        #df_merg.drop(columns=['tok_y', 'sentstart_x', "sentstart_y"], inplace=True)
        df_rep = pd.read_csv(filename_rep, sep="\t").reset_index()
        df_merg = df_merg.merge(right=df_rep, left_index =True, right_on="index")
        df_merg.drop(columns=['tok_x', 'index_y'], inplace=True)
        df_free = pd.read_csv(filename_free, sep="\t").reset_index()
        df_merg = df_merg.merge(right=df_free, left_index =True, right_on="index")
        df_merg.drop(columns=['tok_y', 'sentstart_y', "index_x"], inplace=True)
        
        #print(len(df_merg))
        
        unerwünschte_zeichen = [".", ",", ";", "-", "–", "?", "!", "»", "«", ":"]
        for zeichen in unerwünschte_zeichen:    
            df_merg = df_merg[df_merg.tok_x != zeichen]
        
        #print("bereinigt")
        
        df_merg = df_merg.reset_index()
        
        dir_cnt_konkr = 0
        dir_cnt_abst = 0
        
        narr_cnt_konkr = 0
        narr_cnt_abst = 0
        
        ind_cnt_konkr = 0
        ind_cnt_abst = 0
        
        rep_cnt_konkr = 0
        rep_cnt_abst = 0
        
        free_cnt_konkr = 0
        free_cnt_abst = 0
        
        for j in range(len(df_merg)):
            

            lemma = df_merg["tok_x"].loc[j]
            # Aufgesplittet nach abstrakt/konkret + Redewiedergabe
            
            if df_merg["direct_pred"].loc[j] == "direct":
                if dfca_dict[lemma] != "Not Present":
                    df["abstconc_dir"].loc[i] += dfca_dict[lemma]
                if words_dict[lemma] == "abst":
                    dir_cnt_abst += 1
                elif words_dict[lemma] == "konkr":
                    dir_cnt_konkr += 1
            
            elif df_merg["indirect_pred"].loc[j] == "indirect":
                if dfca_dict[lemma] != "Not Present":
                    df["abstconc_ind"].loc[i] += dfca_dict[lemma]
                if words_dict[lemma] == "abst":
                    ind_cnt_abst += 1
                elif words_dict[lemma] == "konkr":
                    ind_cnt_konkr += 1
            
            elif df_merg["reported_pred"].loc[j] == "reported":
                if dfca_dict[lemma] != "Not Present":
                    df["abstconc_rep"].loc[i] += dfca_dict[lemma]
                if words_dict[lemma] == "abst":
                    rep_cnt_abst += 1
                elif words_dict[lemma] == "konkr":
                    rep_cnt_konkr += 1
                    
            
            elif df_merg["freeIndirect_pred"].loc[j] == "freeIndirect":
                if dfca_dict[lemma] != "Not Present":
                    df["abstconc_free"].loc[i] += dfca_dict[lemma]
                if words_dict[lemma] == "abst":
                    free_cnt_abst += 1
                elif words_dict[lemma] == "konkr":
                    free_cnt_konkr += 1
                    
            else:
                if dfca_dict[lemma] != "Not Present":
                    df["abstconc_narr"].loc[i] += dfca_dict[lemma]
                if words_dict[lemma] == "abst":
                    narr_cnt_abst += 1
                elif words_dict[lemma] == "konkr":
                    narr_cnt_konkr += 1
                
            
        
        all_words_count = len(df_merg)
        
        print("Werte normalisieren")
        #Werte normalisieren
        
        df["ant_narr_abstrakt"].loc[i] = narr_cnt_abst/all_words_count
        df["ant_ind_abstrakt"].loc[i] = ind_cnt_abst/all_words_count
        df["ant_dir_abstrakt"].loc[i] = dir_cnt_abst/all_words_count
        df["ant_rep_abstrakt"].loc[i] = rep_cnt_abst/all_words_count
        df["ant_free_abstrakt"].loc[i] = free_cnt_abst/all_words_count

        df["ant_narr_konkret"].loc[i] = narr_cnt_konkr/all_words_count
        df["ant_ind_konkret"].loc[i] = ind_cnt_konkr/all_words_count
        df["ant_dir_konkret"].loc[i] = dir_cnt_konkr/all_words_count
        df["ant_rep_konkret"].loc[i] = rep_cnt_konkr/all_words_count
        df["ant_free_konkret"].loc[i] = free_cnt_konkr/all_words_count
    
        #Anteil des jeweiligen Redewiedergabe-Typs am Gesamttet
        df["ant_dir_rede"].loc[i] = df_merg['direct_pred'].value_counts()["direct"]/all_words_count
        df["ant_ind_rede"].loc[i] = df_merg['indirect_pred'].value_counts()["indirect"]/all_words_count
        df["ant_rep"].loc[i] = df_merg['reported_pred'].value_counts()["reported"]/all_words_count
        df["ant_free"].loc[i] = df_merg['freeIndirect_pred'].value_counts()["freeIndirect"]/all_words_count
        
        #Werte für einzelne Wortfelder normalisieren
        for wortfeld in wortfelder:
            df["anteil_" + wortfeld].loc[i] = df["anteil_" + wortfeld].loc[i]/all_words_count
                    
        # Abstconc
        df["abstconc"].loc[i] = df["abstconc"].loc[i]/all_words_count
        df["abstconc_dir"].loc[i] = df["abstconc_dir"].loc[i]/all_words_count
        df["abstconc_ind"].loc[i] = df["abstconc_ind"].loc[i]/all_words_count
        df["abstconc_rep"].loc[i] = df["abstconc_rep"].loc[i]/all_words_count
        df["abstconc_free"].loc[i] = df["abstconc_free"].loc[i]/all_words_count
        df["abstconc_narr"].loc[i] = df["abstconc_narr"].loc[i]/all_words_count
                    
    
    else:
        print("Datei fehlt: " + filename_dir.split("/")[2])
    

    

In [ ]:
# speichern als csv-Datei
df_without_lemmas = df.copy()
df_without_lemmas = df_without_lemmas.drop(columns=["all_lemmas"])
df_without_lemmas.to_csv("data/roman_korpus_word_frequencies_redewiedergabe.csv", sep=";")